# Coronavirus Analytics

## Data Preprocessing

In this section we are going to gather our data, gotten from the Johns Hopkins Data Repository at GitHub, and order to cleaning and preparing it for our Power BI dashboard. The processes that were applied include:

* Data formating
* Reshaping
* Handling missing values
* Joins
* New columns calculations

### Data Reading

In [1]:
import numpy as np 
import pandas as pd
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('time_series_covid19_recovered_global.csv')

print("The shape of confirmed is: ", confirmed.shape)
print("The shape of deaths is:    ", deaths.shape)
print("The shape of recovered is: ", recovered.shape)

confirmed.head()

The shape of confirmed is:  (276, 514)
The shape of deaths is:     (276, 514)
The shape of recovered is:  (261, 514)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/5/21,6/6/21,6/7/21,6/8/21,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,77963,79224,80841,82326,84050,85892,87716,88740,89861,91458
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132374,132379,132384,132397,132415,132426,132437,132449,132459,132461
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,130681,130958,131283,131647,132034,132355,132727,133070,133388,133742
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,13758,13758,13777,13781,13791,13805,13813,13813,13813,13826
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,35594,35772,35854,36004,36115,36325,36455,36600,36705,36790


### Reshaping

In [2]:
confirmed_m = pd.melt(confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
deaths_m = pd.melt(deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
recovered_m = pd.melt(recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])


print("The new shape of confirmed is: ", confirmed_m.shape)
print("The new shape of deaths is:    ", deaths_m.shape)
print("The new shape of recovered is: ", recovered_m.shape)

The new shape of confirmed is:  (140760, 6)
The new shape of deaths is:     (140760, 6)
The new shape of recovered is:  (133110, 6)


### Date Column Formating

In [3]:
confirmed_m['Date'] = pd.to_datetime(confirmed_m['Date'])
deaths_m['Date'] = pd.to_datetime(deaths_m['Date'])
recovered_m['Date'] = pd.to_datetime(recovered_m['Date'])

In [4]:
confirmed_m.columns = confirmed_m.columns.str.replace('value', 'Confirmed')
deaths_m.columns = deaths_m.columns.str.replace('value', 'Deaths')
recovered_m.columns = recovered_m.columns.str.replace('value', 'Recovered')

### Handling Missing Values

In [5]:
recovered_m.isnull().sum()

Province/State    97410
Country/Region        0
Lat                 510
Long                510
Date                  0
Recovered             0
dtype: int64

In [6]:
confirmed_m['Province/State'].fillna(confirmed_m['Country/Region'], inplace=True)
deaths_m['Province/State'].fillna(deaths_m['Country/Region'], inplace=True)
recovered_m['Province/State'].fillna(recovered_m['Country/Region'], inplace=True)

confirmed_m.isnull().sum()

Province/State       0
Country/Region       0
Lat               1020
Long              1020
Date                 0
Confirmed            0
dtype: int64

In [7]:
confirmed_m['Lat'].fillna(confirmed_m['Lat'], inplace=True)
deaths_m['Lat'].fillna(deaths_m['Lat'], inplace=True)
recovered_m['Lat'].fillna(recovered_m['Lat'], inplace=True)

confirmed_m['Long'].fillna(confirmed_m['Long'], inplace=True)
deaths_m['Long'].fillna(deaths_m['Long'], inplace=True)
recovered_m['Long'].fillna(recovered_m['Long'], inplace=True)

confirmed_m.isnull().sum()

Province/State       0
Country/Region       0
Lat               1020
Long              1020
Date                 0
Confirmed            0
dtype: int64

### Joining Data

In [8]:
join = confirmed_m.merge(deaths_m[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of first join: ", join.shape)

join = join.merge(recovered_m[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of second join: ", join.shape)

join.head()

Shape of first join:  (140760, 7)
Shape of second join:  (140760, 8)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0


In [9]:
join.isnull().sum()

Province/State       0
Country/Region       0
Lat               1020
Long              1020
Date                 0
Confirmed            0
Deaths               0
Recovered         8160
dtype: int64

In [10]:
full_join = join.dropna(axis=0, how='any')
full_join

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Afghanistan,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
1,Albania,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,Algeria,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,Andorra,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,Angola,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
140755,Vietnam,Vietnam,14.058324,108.277199,2021-06-14,10881,59,4236.0
140756,West Bank and Gaza,West Bank and Gaza,31.952200,35.233200,2021-06-14,311690,3539,304138.0
140757,Yemen,Yemen,15.552727,48.516388,2021-06-14,6863,1350,3765.0
140758,Zambia,Zambia,-13.133897,27.849332,2021-06-14,113134,1416,98994.0


In [11]:
join.isnull().sum()

Province/State       0
Country/Region       0
Lat               1020
Long              1020
Date                 0
Confirmed            0
Deaths               0
Recovered         8160
dtype: int64

In [12]:
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')

In [13]:
test = full_join[full_join['Province/State'] == 'Anhui']
   
full_join2 = test.copy()

full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

full_join3 = test.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'outer',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']

In [14]:
full_join2 = full_join.copy()

full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1',
                          'Date': 'Date Minus 1'}, inplace=True)

full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

full_join3.head()

full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']

print(full_join3.shape)

(132090, 17)


In [15]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Date - 1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


### Deleting Useless Columns

In [16]:
full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']

del full_join3['Confirmed - 1']
del full_join3['Deaths - 1']
del full_join3['Recovered - 1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']

### Dataset Final Result

In [17]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,Jan-2020,0.0,0.0,0.0


In [ ]:
full_join3.to_csv('CoronaVirus.csv', sep='\t')